In [34]:
import dspy
import pandas as pd
import numpy as np
import openai,os,sys
from time import sleep, time
from datetime import date
today = date.today()
from dspy.evaluate import Evaluate
from dspy.teleprompt import MIPRO

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [35]:
seed = 78516
df = pd.read_excel(f'../data/test/lab-manual-split-combine-test-{seed}.xlsx')
df.head()

,index,sentence,year,label,orig_index
0,687,Setting the horizon on the interest rate caps ...,2006,2,666
1,595,"Nonetheless, employment is still 9.5 million b...",1999,0,576
2,824,The shifting balance of domestic demand and po...,2017,1,801
3,133,"By 2009, the forecasts for both the headline a...",2022,2,130
4,297,"In Japan, private consumption rebounded strong...",2006,2,283


In [36]:
train = pd.read_excel(f'../data/train/lab-manual-split-combine-train-{seed}.xlsx')
sample = train.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 10))).reset_index(drop=True)
training_set = [dspy.Example(sentence = sample.loc[i].sentence, answer = sample.loc[i].label).with_inputs("sentence") for i in range(len(sample.index))]
training_set

[Example({'sentence': 'Conditions in the commercial paper (CP) market improved over the intermeeting period, likely reflecting recent measures taken in support of this market, greater demand from institutional investors, and the passing of year-end.', 'answer': 0}) (input_keys={'sentence'}),
 Example({'sentence': 'Most projected somewhat slower growth through next year, and a smaller reduction in unemployment, than they had projected in April.', 'answer': 0}) (input_keys={'sentence'}),
 Example({'sentence': 'The Committee expects that, with appropriate policy accommodation, economic activity will expand at a moderate pace, with labor market indicators continuing to move toward levels the Committee judges consistent with its dual mandate.', 'answer': 0}) (input_keys={'sentence'}),
 Example({'sentence': 'With unemployment still elevated and inflation below the Committee’s longer-run objective, the Committee is continuing its highly accommodative policies.', 'answer': 0}) (input_keys={'se

In [37]:
testing_set = [dspy.Example(sentence = df.loc[i].sentence, answer = df.loc[i].label).with_inputs("sentence") for i in range(len(df))]
testing_set

[Example({'sentence': 'Setting the horizon on the interest rate caps to reinforce forward guidance on the policy rate would augment the credibility of the yield curve caps and thereby diminish concerns about an open-ended balance sheet commitment.', 'answer': 2}) (input_keys={'sentence'}),
 Example({'sentence': 'Nonetheless, employment is still 9.5 million below its pre-pandemic level for the economy as a whole.', 'answer': 0}) (input_keys={'sentence'}),
 Example({'sentence': 'The shifting balance of domestic demand and potential supply in each country means that policies affecting domestic demand will need to be re-calibrated to preserve price stability and keep economies operating at high levels of reserve utilization.', 'answer': 1}) (input_keys={'sentence'}),
 Example({'sentence': 'By 2009, the forecasts for both the headline and core PCE price indexes showed inflation receding from its 2008 level, in line with the previous forecasts.', 'answer': 2}) (input_keys={'sentence'}),
 Exa

In [38]:
len(training_set)

30

In [39]:
sent = df.loc[0].sentence
print(sent)

Setting the horizon on the interest rate caps to reinforce forward guidance on the policy rate would augment the credibility of the yield curve caps and thereby diminish concerns about an open-ended balance sheet commitment.


In [40]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=1000, api_key=api_key)
dspy.settings.configure(lm=turbo)
d = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}

In [41]:
class StanceAnalysis(dspy.Signature):
    """Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish."""
    
    sentence = dspy.InputField()
    stance = dspy.OutputField(desc = "hawkish, neutral, or dovish")

class Analysis(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(StanceAnalysis)
    
    def forward(self, sentence):
        return self.predict(sentence=sentence)

In [42]:
classify = dspy.Predict(StanceAnalysis)
classify(sentence=sent)

analyze = Analysis()
analyze(df.loc[4].sentence)

Prediction(
    stance='hawkish'
)

In [43]:
df.loc[4]

index                                                       297
sentence      In Japan, private consumption rebounded strong...
year                                                       2006
label                                                         2
orig_index                                                  283
Name: 4, dtype: object

In [44]:
def answer_match(example, pred, trace=None):
    answer_match = d[example.answer].lower() == pred.stance.lower()
    # print(f"Actual: {d[example.answer].lower()}, Predicted: {pred.stance.lower()}")
    return answer_match

In [14]:
evaluate = Evaluate(devset=training_set, metric=answer_match, display_progress=True, display_table=0)
evaluate(Analysis())

Average Metric: 0 / 1  (0.0):   3%|▎         | 1/30 [00:00<00:09,  3.17it/s]

Actual: dovish, Predicted: neutral


Average Metric: 1 / 2  (50.0):   7%|▋         | 2/30 [00:00<00:10,  2.80it/s]

Actual: dovish, Predicted: dovish


Average Metric: 1 / 3  (33.3):  10%|█         | 3/30 [00:00<00:08,  3.30it/s]

...

Average Metric: 21 / 29  (72.4):  97%|█████████▋| 29/30 [00:08<00:00,  3.62it/s]

Actual: neutral, Predicted: neutral


Average Metric: 21 / 30  (70.0): 100%|██████████| 30/30 [00:09<00:00,  3.17it/s]

Actual: neutral, Predicted: dovish
Average Metric: 21 / 30  (70.0%)


70.0

In [45]:
teleprompter = MIPRO(metric = answer_match)
optimized_program = teleprompter.compile(Analysis(), trainset = training_set, num_trials=5, max_bootstrapped_demos=3, max_labeled_demos=3, eval_kwargs=dict(display_progress=True, display_table=0))


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 30 examples in dev set * 5 trials * # of LM calls in your program = (150 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 10 * 1 lm calls in program = 20 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:

- Re

 10%|█         | 3/30 [00:01<00:11,  2.34it/s]


Bootstrapped 3 full traces after 4 examples in round 0.


 10%|█         | 3/30 [00:00<00:06,  4.26it/s]


Bootstrapped 3 full traces after 4 examples in round 0.


...

Average Metric: 11 / 30  (36.7%)
Starting trial #3


Average Metric: 19 / 30  (63.3): 100%|██████████| 30/30 [00:09<00:00,  3.18it/s]
[I 2024-04-07 15:24:23,116] Trial 3 finished with value: 63.33 and parameters: {'2170764072224_predictor_instruction': 9, '2170764072224_predictor_demos': 3}. Best is trial 3 with value: 63.33.


Average Metric: 19 / 30  (63.3%)
Starting trial #4


Average Metric: 19 / 30  (63.3): 100%|██████████| 30/30 [00:09<00:00,  3.33it/s]
[I 2024-04-07 15:24:32,130] Trial 4 finished with value: 63.33 and parameters: {'2170764072224_predictor_instruction': 8, '2170764072224_predictor_demos': 4}. Best is trial 3 with value: 63.33.


Average Metric: 19 / 30  (63.3%)
Returning predict = Predict(StringSignature(sentence -> stance
    instructions="classify the sentence's stance on the monetary policy between supportive, neutral, and opposing."
    sentence = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Sentence:', 'desc': '${sentence}'})
    stance = Field(annotation=str required=True json_schema_extra={'desc': 'hawkish, neutral, or dovish', '__dspy_field_type': 'output', 'prefix': '[Classification]", followed by the suggested stance for the prompt, for example: "[Classification] hawkish", "[Classification] neutral", or "[Classification] dovish'})
)) from continue_program


In [46]:
optimized_program

predict = Predict(StringSignature(sentence -> stance
    instructions="classify the sentence's stance on the monetary policy between supportive, neutral, and opposing."
    sentence = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Sentence:', 'desc': '${sentence}'})
    stance = Field(annotation=str required=True json_schema_extra={'desc': 'hawkish, neutral, or dovish', '__dspy_field_type': 'output', 'prefix': '[Classification]", followed by the suggested stance for the prompt, for example: "[Classification] hawkish", "[Classification] neutral", or "[Classification] dovish'})
))

In [47]:
evaluate = Evaluate(devset=testing_set, metric=answer_match, display_progress=True, return_outputs=True)
outputs = evaluate(optimized_program)

Average Metric: 323 / 496  (65.1): 100%|██████████| 496/496 [02:32<00:00,  3.26it/s]

Average Metric: 323 / 496  (65.1%)


In [48]:
outdf = pd.DataFrame(columns=['Sentence', 'Actual', 'Predicted'])
for pred in outputs[1]:
    outdf.loc[len(outdf)] = [pred[0].sentence, d[pred[0].answer].lower(), pred[1].stance.lower()]
outdf.to_csv(f"../data/llm_prompt_outputs/dspy_{seed}.csv", index=False)
print(f"Accuracy: {accuracy_score(outdf['Actual'], outdf['Predicted'])}")
print(f"F1: {f1_score(outdf['Actual'], outdf['Predicted'], average='weighted')}")
outdf.head()

Accuracy: 0.6512096774193549
F1: 0.6398555274413625


,Sentence,Actual,Predicted
0,Setting the horizon on the interest rate caps ...,neutral,neutral
1,"Nonetheless, employment is still 9.5 million b...",dovish,dovish
2,The shifting balance of domestic demand and po...,hawkish,neutral
3,"By 2009, the forecasts for both the headline a...",neutral,neutral
4,"In Japan, private consumption rebounded strong...",neutral,neutral


In [49]:
print(np.mean([0.6068066709, 0.6398555274, 0.6768750959]))
print(np.std([0.6068066709, 0.6398555274, 0.6768750959]))

0.6411790980666666
0.028620621062180572
